In [12]:
def get_predict(h):
    pred = {}
    for w,n, in h.items():
        for i in range(1,len(w)):
            a = w[:i]
            if n > h.get(pred.get(a,None),0):
                pred[a] = w
    return pred
                
h = {"ez": 30, "erre": 1, "eraman": 8}
get_predict(h)    

{'e': 'ez',
 'er': 'eraman',
 'err': 'erre',
 'era': 'eraman',
 'eram': 'eraman',
 'erama': 'eraman'}